# Create album

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import *    
import os 
import pyreadr
import numpy as np
import pyreadr
import pandas as pd
import butterfly.album
import butterfly.CNN
from itertools import combinations 
import multiprocessing
from joblib import Parallel, delayed

Using TensorFlow backend.


In [2]:
#Import the data
DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
#DF = pyreadr.read_r('/home/mxenoc/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [3]:
# Choose your predicted omics, size for the image and number of features we are predicting
response = 'serum_l'
pixels = 40
features = 62

response = [col for col in DF if col.startswith(response)]
response.append("patientID")
response.append("trimester")    
response_df = DF[response]

yy = response_df.drop(['patientID', 'trimester'], axis =1 ).values

#Make sure your B trimesters are properly converted to 4

In [4]:
#Get every combination of patient IDs in pairs of m's
# Function which returns subset or r length from n 
m = 2
IDs = response_df['patientID'].unique()
IDs = IDs.tolist()
exclude = list(combinations(IDs, m))

In [5]:
predictors = ['rna', 'plasma_l', 'microb', 'immune', 'metabol', 'plasma_s']

In [ ]:
albums = Parallel(n_jobs=3, backend="threading")(delayed(butterfly.album.create_album)
                                                (DF, predictors[al], pixels) 
                                                for al in range(6))

In [ ]:
prediction, observed = Parallel(n_jobs=3, backend="threading")(delayed(butterfly.CNN.model_multi)
                                                               (response_df, exclude, cv, albums, pixels) 
                                                               for cv in range(len(exclude)))

In [ ]:
#Create your CNN
model = Sequential()
model.add(Conv2D(32,(3,3), input_shape=(pixels, pixels,6)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(features))
model.add(Activation( 'sigmoid'))
model.compile(optimizer='adam', loss='mse')

model.fit(X_cc, y_c, epochs=300, verbose=0)

# demonstrate prediction
y_pred = model.predict(X_vv, verbose = 0)
y_pred = pd.DataFrame(y_pred)
prediction.append(y_pred)
observed.append(y_v)

In [ ]:
cv = 0
album = albums 

yy = response_df.drop(['patientID', 'trimester'], axis =1 ).values

#Create your calibration and validation datasets
pt1ex = response_df.index[response_df['patientID'] == exclude[cv][0]].tolist()
pt2ex = response_df.index[response_df['patientID'] == exclude[cv][1]].tolist()
ptex = pt1ex+pt2ex

X_c = []
X_v = []

#Divide in calibration and validation
for i in range(6):
    X_cN = np.delete(album[i], ptex, 0)
    X_cN = X_cN.reshape((X_cN.shape[0], X_cN.shape[1], pixels))
    X_c.append(X_cN)   

    X_vN = np.asarray([album[i][k]  for k in ptex])
    X_vN = X_vN.reshape((X_vN.shape[0], X_vN.shape[1], pixels))
    X_v.append(X_vN)   

y_c = np.delete(yy, ptex, 0)
y_c = pd.DataFrame(StandardScaler().fit_transform(y_c))

X_cc = []



In [ ]:
for i in range(60):
    X_cc.append(np.array((X_c[0][i], X_c[1][i], X_c[2][i], X_c[3][i], X_c[4][i], 
                          X_c[5][i]), dtype=float))



In [ ]:
X_cc = np.array(X_cc)
X_cc = X_cc.reshape((X_cc.shape[0], pixels, pixels, X_cc.shape[1]))



In [ ]:
y_v = np.asarray([yy[i] for i in ptex])
y_v = pd.DataFrame(StandardScaler().fit_transform(y_v))



In [ ]:
X_vv = []

for i in range(8):
    X_vv.append(np.array((X_v[0][i], X_v[1][i], X_v[2][i], X_v[3][i], X_v[4][i], 
                         X_v[5][i]), dtype=float))



In [ ]:
X_v[0].shape

In [ ]:
X_vv = np.array(X_vv)
X_vv = X_vv.reshape((X_vv.shape[0], pixels, pixels, X_vv.shape[1]))

In [ ]:
#Create your CNN
model = Sequential()
model.add(Conv2D(32,(3,3), input_shape=(pixels, pixels,6)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(features))
model.add(Activation( 'sigmoid'))
model.compile(optimizer='adam', loss='mse')

model.fit(X_c, y_c, epochs=300, verbose=0)

In [ ]:




# demonstrate prediction
y_pred = model.predict(X_v, verbose = 0)
y_pred = pd.DataFrame(y_pred)
prediction.append(y_pred)
observed.append(y_v)

In [ ]:
correl = prediction.corrwith(observed, axis = 0) 
correl[correl < 0] = 0
np.mean(correl)

In [45]:
#Create your CNN
model = Sequential()
model.add(Conv2D(32,(3,3), input_shape=(pixels, pixels,6)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(features))
model.add(Activation( 'sigmoid'))
model.compile(optimizer='adam', loss='mse')

model.fit(X_cc, y_c, epochs=300, verbose=0)

# demonstrate prediction
y_pred = model.predict(X_vv, verbose = 0)
y_pred = pd.DataFrame(y_pred)
prediction.append(y_pred)
observed.append(y_v)

AttributeError: 'list' object has no attribute 'shape'

In [31]:
cv = 0
album = albums 

yy = response_df.drop(['patientID', 'trimester'], axis =1 ).values

#Create your calibration and validation datasets
pt1ex = response_df.index[response_df['patientID'] == exclude[cv][0]].tolist()
pt2ex = response_df.index[response_df['patientID'] == exclude[cv][1]].tolist()
ptex = pt1ex+pt2ex

X_c = []
X_v = []

#Divide in calibration and validation
for i in range(6):
    X_cN = np.delete(album[i], ptex, 0)
    X_cN = X_cN.reshape((X_cN.shape[0], X_cN.shape[1], pixels))
    X_c.append(X_cN)   

    X_vN = np.asarray([album[i][k]  for k in ptex])
    X_vN = X_vN.reshape((X_vN.shape[0], X_vN.shape[1], pixels))
    X_v.append(X_vN)   

y_c = np.delete(yy, ptex, 0)
y_c = pd.DataFrame(StandardScaler().fit_transform(y_c))

X_cc = []



In [32]:
for i in range(60):
    X_cc.append(np.array((X_c[0][i], X_c[1][i], X_c[2][i], X_c[3][i], X_c[4][i], 
                          X_c[5][i]), dtype=float))



In [33]:
X_cc = np.array(X_cc)
X_cc = X_cc.reshape((X_cc.shape[0], pixels, pixels, X_cc.shape[1]))



In [34]:
y_v = np.asarray([yy[i] for i in ptex])
y_v = pd.DataFrame(StandardScaler().fit_transform(y_v))



In [35]:
X_vv = []

for i in range(8):
    X_vv.append(np.array((X_v[0][i], X_v[1][i], X_v[2][i], X_v[3][i], X_v[4][i], 
                         X_v[5][i]), dtype=float))



In [36]:
X_v[0].shape

(8, 40, 40)

In [37]:
X_vv = np.array(X_vv)
X_vv = X_vv.reshape((X_vv.shape[0], pixels, pixels, X_vv.shape[1]))

In [39]:
#Create your CNN
model = Sequential()
model.add(Conv2D(32,(3,3), input_shape=(pixels, pixels,6)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(features))
model.add(Activation( 'sigmoid'))
model.compile(optimizer='adam', loss='mse')

model.fit(X_c, y_c, epochs=300, verbose=0)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 6 arrays: [array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., ...

In [38]:




# demonstrate prediction
y_pred = model.predict(X_v, verbose = 0)
y_pred = pd.DataFrame(y_pred)
prediction.append(y_pred)
observed.append(y_v)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 6 arrays: [array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., ...

In [ ]:
correl = prediction.corrwith(observed, axis = 0) 
correl[correl < 0] = 0
np.mean(correl)